In [1]:
import numpy as np 
import pandas as pd 
from scipy import stats
import calendar
import matplotlib as plt
import category_encoders as ce
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction import FeatureHasher

In [2]:
def encode(x): return 1 if x == 'M' else 0

In [4]:
def binning(df,colName,labels):
    #Define min and max values:
    #create list by adding min and max to cut_points
    break_points = [df[colName].min()] + [df[colName].describe()[4]] + [df[colName].describe()[5]] + [df[colName].describe()[6]] + [df[colName].max()]

    #labels = ["low","medium","high","very high"]

    #Binning using cut function of pandas
    colBin = pd.cut(df[colName],bins=break_points,labels=labels,include_lowest=True)
    return colBin

In [21]:
df = pd.read_csv('data/train.csv')
labels = ["young","adult","old","elder"]
df['Date']=pd.to_datetime(df['Date'], format='%Y%m', errors='ignore')
df=df.sort_values('Date')
df['Month'] = df['Date'].dt.month.apply(lambda x: calendar.month_abbr[x])
df['Gender'] = df.Gender.apply(encode)
df['Age_bin'] = binning(df,"Age",labels)
df.head(10)

,UID,Age,Gender,Date,Event_Code,Month,Age_bin
672310,Id_e45ad28b,13,0,2011-01-01,2446,Jan,young
436052,Id_e45ad2d0,72,1,2011-01-01,A425,Jan,elder
109925,Id_e45b478b,51,1,2011-01-01,8780,Jan,old
644917,Id_e45bbc2f,72,0,2011-01-01,2946,Jan,elder
746643,Id_e45aab7d,23,1,2011-01-01,5990,Jan,young
339333,Id_e45a8479,15,1,2011-01-01,V458,Jan,young
409055,Id_e45be32c,73,1,2011-01-01,3082,Jan,elder
94614,Id_e45b4770,73,1,2011-01-01,3641,Jan,elder
370953,Id_e45aabaa,72,1,2011-01-01,8573,Jan,elder
477867,Id_e45b95b1,11,0,2011-01-01,9921,Jan,young


In [18]:
patientTab = pd.crosstab(df['Event_Code'],df['Age_bin'])
#print(patientTab)
stats.chi2_contingency(patientTab,correction=False)

(88674.36567427745,
 0.0,
 19413,
 array([[1.00282347, 1.00839477, 1.00383027, 0.98495149],
        [0.25070587, 0.25209869, 0.25095757, 0.24623787],
        [1.75494107, 1.76469085, 1.75670297, 1.72366511],
        ...,
        [0.50141173, 0.50419738, 0.50191513, 0.49247575],
        [0.25070587, 0.25209869, 0.25095757, 0.24623787],
        [2.75776454, 2.77308562, 2.76053324, 2.7086166 ]]))

In [13]:
X = df.loc[:,['Gender','Month']]
y = df.loc[:,['Event_Code']]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.33)
lr = linear_model.LinearRegression()
model = lr.fit(X_train, y_train)

ValueError: could not convert string to float: 'J301'